In [12]:
import pandas as pd
from datetime import datetime, timedelta
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import glob

In [13]:
#Processes a single wide-format MISO Day-Ahead EPNode CSV and returns a long-format DataFrame with datetime, node, and LMP components.
#Data Source: https://www.misoenergy.org/markets-and-operations/real-time--market-data/market-reports/#nt=%2FMarketReportType%3AHistorical%20LMP%2FMarketReportName%3ADay-Ahead%20EPNode%20LMP%20(zip)&t=10&p=3&s=MarketReportPublished&sd=desc
def process_wide_format_safely(file_path):

    # Step 1: Read lines and find header row
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Locate header row (should contain "EPNode" and "HE1")
    header_row_index = next(
        (i for i, line in enumerate(lines) if "EPNode" in line and "HE1" in line),
        None
    )

    if header_row_index is None:
        raise ValueError("Could not find valid header row in file.")

    # Step 2: Extract market date (e.g., "Market Day: 06/05/2025")
    market_day_line = next(
        (line for line in lines if "Market Day" in line),
        None
    )
    if not market_day_line:
        raise ValueError("Market Day not found in file.")

    market_day_str = market_day_line.split(":")[-1].strip()
    market_date = datetime.strptime(market_day_str, "%m/%d/%Y")

    # Step 3: Load CSV from the correct header row
    df = pd.read_csv(file_path, skiprows=header_row_index)

    # Step 4: Convert wide to long format
    long_data = []

    for i in range(0, len(df) - 2, 3):  # Process 3 rows at a time (LMP, MCC, MLC)
        try:
            row_lmp = df.iloc[i]
            row_mcc = df.iloc[i + 1]
            row_mlc = df.iloc[i + 2]

            # Safe access to first column (node name), drop leading "L"
            node = " ".join(str(row_lmp.iloc[0]).split()[1:])

            for hour in range(1, 25):  # HE1 to HE24
                hour_col = f"HE{hour}"
                dt = market_date + timedelta(hours=hour - 1)

                long_data.append({
                    "datetime": dt,
                    "node": node,
                    "lmp_total": float(row_lmp[hour_col]),
                    "congestion": float(row_mcc[hour_col]),
                    "loss": float(row_mlc[hour_col])
                })

        except Exception as e:
            print(f"Error processing rows {i}-{i+2}: {e}")
            continue

    return pd.DataFrame(long_data)

In [14]:
dirty_folder = "data/dirty/DA/Wide"
cleaned_folder = "data/dirty/DA/Long"

os.makedirs(cleaned_folder, exist_ok=True)  # make sure cleaned folder exists
file_path = 'data/cleaned/DA/miso_da_combined_clean.csv'

if not os.path.exists(file_path):
    print(f"The file {file_path} does not exist. Running the processing code...")
    for filename in sorted(os.listdir(dirty_folder)):
        if filename.startswith("DA_Load_EPNodes") and filename.endswith(".csv"):
            input_path = os.path.join(dirty_folder, filename)
            output_path = os.path.join(cleaned_folder, filename.replace("DA_Load_EPNodes", "miso_da")[:-4] + "_clean.csv")
            
            print(f"Processing {filename} ...")
            try:
                df = process_wide_format_safely(input_path)
                df.to_csv(output_path, index=False)
                print(f"Saved cleaned file to {output_path}")
            except Exception as e:
                print(f"Failed to process {filename}: {e}")
else:
    print(f"The file {file_path} already exists. Skipping processing.")



The file data/cleaned/DA/miso_da_combined_clean.csv already exists. Skipping processing.


In [15]:
def combine_cleaned_csvs(folder="data/dirty/DA/Long", prefix="miso_da_", ext=".csv"):
    all_dfs = []
    
    for filename in sorted(os.listdir(folder)):
        if filename.startswith(prefix) and filename.endswith(ext):
            file_path = os.path.join(folder, filename)
            print(f"📄 Loading: {filename}")
            try:
                df = pd.read_csv(file_path, parse_dates=["datetime"])
                all_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to load {filename}: {e}")

    if not all_dfs:
        print("❌ No files combined.")
        return pd.DataFrame()

    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.sort_values(by=["datetime", "node"], inplace=True)
    combined_df.to_csv("data/cleaned/DA/miso_da_combined_clean.csv")
    print(f"✅ Combined dataframe saved")
    return 

In [16]:
file_path = 'data/cleaned/DA/miso_da_combined_clean.csv'

if not os.path.exists(file_path):
    print(f"The file {file_path} does not exist. Running the processing code...")
    combine_cleaned_csvs()
    # Make sure the directory exists before saving
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    print(f"Processing complete. File saved to {file_path}")
else:
    print(f"The file {file_path} already exists. Skipping processing.")

The file data/cleaned/DA/miso_da_combined_clean.csv already exists. Skipping processing.


In [17]:
shadow_prices = pd.read_csv('data/cleaned/MPMA clearing/20250501_ftr_mpma_results/SourceSinkShadowPrices_May25_AUCTION_May25Auc_Round_1.csv')
DA_LMP = pd.read_csv('data/cleaned/DA/miso_da_combined_clean.csv', index_col = 0)

In [18]:
# LMP nodes (your nodes to match)
lmp_nodes = list(set(DA_LMP['node']))

# SourceSink nodes from FTR auction paths
source_sink_nodes = list(set(shadow_prices['SourceSink']))

In [19]:
# Create a DataFrame to store results
results = []

for lmp in lmp_nodes:
    extraction_result = process.extractOne(lmp, source_sink_nodes, scorer=fuzz.token_sort_ratio)
    match = extraction_result[0]  # First element is the matched string
    score = extraction_result[1]  # Second element is the score
    
    results.append({
        'lmp_node': lmp,
        'matched_source_sink': match,
        'match_score': score
    })

matched_df = pd.DataFrame(results)

In [20]:
file_path = 'Outputs/LMP_FTR_Mapping.csv'

if not os.path.exists(file_path):
    print(f"The file {file_path} does not exist. Running the processing code...")
    matched_df.to_csv('Outputs/LMP_FTR_Mapping.csv')    # Make sure the directory exists before saving
    print(f"Processing complete. File saved to {file_path}")
else:
    print(f"The file {file_path} already exists. Skipping processing.")
    

The file Outputs/LMP_FTR_Mapping.csv already exists. Skipping processing.
